In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 

import xgboost
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline

from collections import defaultdict

import optuna

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.metrics import brier_score_loss, roc_auc_score, accuracy_score, confusion_matrix, classification_report


In [ ]:
odds_df = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\upcoming_odds.csv')
stats_df = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\new_combined.csv')
upcoming_file_path = r'C:\Users\jcmar\my_files\SportsBetting\data\next_event_features.csv'
upcoming_df = pd.read_csv(upcoming_file_path)

full_file_path = r'C:\Users\jcmar\my_files\SportsBetting\data\odds_stats.csv'
full_df = pd.read_csv(full_file_path)
print(upcoming_df.shape, stats_df.shape, odds_df.shape, stats_df.shape, full_df.shape)

In [ ]:
model_cols = [
                'total_td_landed_diff', 'total_damage_diff','total_strikes_landed_diff','total_td_defended_diff','num_fights_diff',
                'open_red', 'open_blue', 'close1_red','close2_red', 'close1_blue', 'close2_blue', 
                'ratio_td_red', 'ratio_td_blue','ratio_sigstrike_red', 'ratio_sigstrike_blue','ratio_control_red', 'ratio_control_blue',
                'total_landed_red', 'total_landed_blue',
                'total_damage_red','total_damage_blue','womens_fight',
                'win_pct_red','win_pct_blue','num_fights_blue','num_fights_red',
                'red_td_landed', 'blue_td_landed', 'red_td_defended', 'blue_td_defended',

                'months_since_red', 'months_since_blue','kd_pr_red','kd_pr_blue', 'reverse_pr_red', 'reverse_pr_blue',
            
                'control_pr_red', 'control_pr_blue', 
                'td_pr_red','td_pr_blue', 
                'red_td_accuracy','blue_td_accuracy',
                'red_td_defense','blue_td_defense',
                'sub_att_pr_red','sub_att_pr_blue',

                'red_kd_total', 'blue_kd_total',
                'sigstrikes_pm_red','sigstrikes_pm_blue',
                'red_sigstrike_accuracy','blue_sigstrike_accuracy',
                'red_sigstrike_defense','blue_sigstrike_defense',
                
                'sigstrikes_pm_red', 'sigstrikes_pm_blue',
                'leg_strikes_pm_red','leg_strikes_pm_blue',
                'body_strikes_pm_red', 'body_strikes_pm_blue',
                'head_strikes_pm_red','head_strikes_pm_blue',
                'clinch_strikes_pm_red', 'clinch_strikes_pm_blue',
                'ground_strikes_pm_red', 'ground_strikes_pm_blue',
                'sigstrikes_absorbed_pm_red','sigstrikes_absorbed_pm_blue',

                'reach_diff', 'age_diff',
                'reach_red','reach_blue','weight_class',
                'height_red', 'height_blue', 
                'red_age', 'blue_age', 
                
                'red_elo','blue_elo', 'elo_diff',
                'math_red','math_blue',

                'red_win_streak','blue_win_streak',
                'red_lose_streak','blue_lose_streak',
                'ko_wins_red','ko_wins_blue',
                'sub_wins_red','sub_wins_blue',
                'decision_wins_blue','decision_wins_red',

                'winner', 'red_fighter','blue_fighter',

                'sub_att_pr_diff', 'sigstrikes_pm_diff', 'head_strikes_pm_diff', 'leg_strikes_pm_diff',
                'body_strikes_pm_diff', 'clinch_strikes_pm_diff', 'ground_strikes_pm_diff', 'kd_pr_diff',
                'reverse_pr_diff', 'sigstrikes_absorbed_pm_diff', 'control_pr_diff', 'td_pr_diff', 
                'sub_att_pr_diff', 'sigstrikes_pm_diff', 'td_defense_diff', 'sig_strike_defense_diff',
                'td_accuracy_diff', 'sig_strike_accuracy_diff',
                'red_glicko', 'blue_glicko', 'glicko_diff', 'red_glicko_rd', 'blue_glicko_rd', 'glicko_rd_diff',
                'date'
                ]
#removed sig_strike_attempted
full_df[model_cols]

In [ ]:

xgb_dat = full_df[model_cols].dropna()
# xgb_dat = xgb_dat.dropna()
print('AFTER NA SHAPE: ', xgb_dat.shape)

le = LabelEncoder()
# xgb_dat['event_country'] = le.fit_transform(xgb_dat['event_country'])
xgb_dat['weight_class'] = le.fit_transform(xgb_dat['weight_class'])

mm_scaler = MinMaxScaler()
cat_cols = xgb_dat.select_dtypes(include=['object', 'category']).columns.tolist()
num_categorical = xgb_dat.select_dtypes(include=['object']).shape[1]
print(f"Number of categorical columns: {num_categorical}")
numerical_cols = xgb_dat.columns.difference(cat_cols)
xgb_dat[numerical_cols] = mm_scaler.fit_transform(xgb_dat[numerical_cols])

train_len = int(xgb_dat.shape[0] * .85)

train_dat = xgb_dat.iloc[:train_len]
test_dat = xgb_dat.iloc[train_len:]

X_train = train_dat.drop(columns=['winner','red_fighter','blue_fighter','date'])
y_train = train_dat['winner']

X_test = test_dat.drop(columns=['winner','red_fighter','blue_fighter','date'])
y_test= test_dat['winner']


no_na = xgb_dat.dropna()
mm_scaler = MinMaxScaler()
le = LabelEncoder()
no_na['weight_class'] = le.fit_transform(no_na['weight_class'])
cat_cols = no_na.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols.append('date')

# Number of categorical columns (excluding 'date' if it's not object type)
num_categorical = no_na.select_dtypes(include=['object']).shape[1]
print(f"Number of categorical columns: {num_categorical}")

# Get all other columns (numerical, etc.)
numerical_cols = no_na.columns.difference(cat_cols)
drop_vals = ['open_red', 'open_blue','winner_red_blue']
numerical_cols = [x for x in numerical_cols if x not in drop_vals]
# Scale numerical columns
no_na[numerical_cols] = mm_scaler.fit_transform(no_na[numerical_cols])
train_len_drop = int(no_na.shape[0] * .9)
train_dat_drop = no_na.iloc[:train_len_drop]
test_dat_drop = no_na.iloc[train_len_drop:]
y_train_drop =  no_na.iloc[:train_len_drop]['winner']
y_test_drop= no_na.iloc[train_len_drop:]['winner']

X_train_drop = train_dat_drop.drop(columns=['winner','red_fighter','blue_fighter'])
X_test_drop = test_dat_drop.drop(columns=['winner','red_fighter','blue_fighter'])

In [ ]:
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
#  Params:  {'lambda': 0.002897710738167055, 'alpha': 0.0001149053240987921, 
# 'colsample_bytree': 0.7004457760139489, 'subsample': 0.6482734775091901, 
# 'learning_rate': 0.016034177246955644,
#  'max_depth': 3, 'n_estimators': 2343, 'min_child_weight': 7, 'gamma': 7.586270218472457}


xgb = xgboost.XGBClassifier(
    objective="binary:logistic", 
    eval_metric="aucpr",  
    reg_lambda = 0.002897710738167055,
    reg_alpha = .0001149053240987921,
    colsample_bytree = 0.7004457760139489,
    subsample = 0.6482734775091901,
    learning_rate = 0.016034177246955644,
    max_depth = 3,
    n_estimators = 2343,
    min_child_weight = 7,
    use_label_encoder = False,
    gamma =  7.586270218472457,
    scale_pos_weight = scale_pos_weight
)
X_train = np.array(X_train)
X_test = np.array(X_test)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy Test: {accuracy}')

y_pred_test = xgb.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_test)
print(f'Accuracy Train: {accuracy_train}')

# xgboost.plot_importance(xgb)
probability_scores_1 = xgb.predict_proba(X_test)

cm = confusion_matrix(y_test, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Blue Wins", "Red Wins"], yticklabels=["Blue Wins", "Red Wins"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

print(classification_report(y_test, y_pred))

drop_columns = ('winner','red_fighter','blue_fighter')

column_names = [c for c in model_cols if c not in drop_columns]
xgb.get_booster().feature_names = column_names
fig, ax = plt.subplots(figsize=(12, 10)) 

xgboost.plot_importance(xgb, ax=ax) 


In [ ]:
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])


# {'lambda': 0.0015603738680954077, 'alpha': 7.484955255987673e-05, 'colsample_bytree': 0.4549059715922109, 
# 'subsample': 0.5180453069382756, 'learning_rate': 0.006855575249709778,
#  'max_depth': 15, 'n_estimators': 1991,
#  'min_child_weight': 3, 'gamma': 0.7694919023951203}

xgb = xgboost.XGBClassifier(
    objective="binary:logistic", 
    eval_metric="aucpr",  
    reg_lambda =   0.0015603738680954077,
    reg_alpha = 7.484955255987673e-05,
    colsample_bytree =   0.4549059715922109,
    subsample =  0.5180453069382756,
    learning_rate = 0.006855575249709778,
    max_depth = 3,
    n_estimators = 1991,
    min_child_weight = 3,
    use_label_encoder = False,
    gamma =  0.7694919023951203,
    scale_pos_weight = scale_pos_weight
)
X_train = np.array(X_train)
# X_test[~np.isnan(X_test).any(axis=1)]
# X_test = np.array(X_test)
# X_test = X_test[~np.isnan(X_test).any(axis=1)]
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy Test: {accuracy}')

y_pred_test = xgb.predict(X_train)
accuracy_train = accuracy_score(y_train, y_pred_test)
print(f'Accuracy Train: {accuracy_train}')

# xgboost.plot_importance(xgb)
probability_scores_1_train = xgb.predict_proba(X_train)
probability_scores_1_test = xgb.predict_proba(X_test)

cm = confusion_matrix(y_test, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Blue Wins", "Red Wins"], yticklabels=["Blue Wins", "Red Wins"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

print(classification_report(y_test, y_pred))

drop_columns = ('winner','red_fighter','blue_fighter')

column_names = [c for c in model_cols if c not in drop_columns]
xgb.get_booster().feature_names = column_names
fig, ax = plt.subplots(figsize=(12, 10)) 

xgboost.plot_importance(xgb, ax=ax) 


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
logreg_train_prob = model.predict_proba(X_train)
logreg_test_prob = model.predict_proba(X_test)

cm = confusion_matrix(y_test, y_pred)

# Plot Confusion Matrix
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Blue Wins", "Red Wins"], yticklabels=["Blue Wins", "Red Wins"])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))